# Calculating Potential Customer Returns with Rule-Based Classification

## Business Problem

A gaming company wants to create level-based new customer profiles (personas) using certain characteristics of its customers and then create segments based on these new customer profiles. They aim to estimate how much, on average, new potential customers, who match these segments, could potentially bring in revenue for the company.

*The dataset "Persona.csv" contains the prices of products sold by an international gaming company and some demographic information of the users who purchased these products. The dataset is composed of records generated for each sales transaction. This means that the table has not been deduplicated. In other words, a user with specific demographic characteristics may have made multiple purchases.*

## Variables

- PRICE: Customer Expenditure

- SOURCE: Customer's Connected Device Type

- SEX 

- COUNTRY

- AGE

In [1]:
import pandas as pd
pd.set_option("display.max_rows", None)

We first read the persona.csv file and check the general information about the dataset.

In [2]:
df = pd.read_csv("/kaggle/input/persona-csv/persona.csv")

In [3]:
df.head(5)
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


How many unique SOURCE values are there? What are their frequencies?"

In [4]:
df["SOURCE"].nunique()
df["SOURCE"].value_counts(normalize=True)

SOURCE
android    0.5948
ios        0.4052
Name: proportion, dtype: float64

How many unique PRICE values are there?

In [5]:
df["PRICE"].nunique()

6

In [6]:
df["PRICE"].value_counts()

PRICE
29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: count, dtype: int64

Which country has how many sales?

In [7]:
df.pivot_table(values="PRICE", index="COUNTRY", aggfunc="count")

,PRICE
COUNTRY,
bra,1496
can,230
deu,455
fra,303
tur,451
usa,2065


Total revenue from sales by country

In [8]:
df.pivot_table(values="PRICE", index="COUNTRY", aggfunc=["sum"])

,sum
,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


The number of sales by SOURCE types

In [9]:
df.groupby("SOURCE").agg({"PRICE": ["count"]})

,PRICE
,count
SOURCE,
android,2974
ios,2026


The PRICE averages by country

In [10]:
df.groupby("COUNTRY").agg({"PRICE": ["mean"]})

,PRICE
,mean
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


The PRICE averages by SOURCE

In [11]:
df.groupby('SOURCE').agg({"PRICE": ["mean"]})

,PRICE
,mean
SOURCE,
android,34.174849
ios,34.069102


The PRICE averages by COUNTRY-SOURCE grouping

In [12]:
df.groupby(["COUNTRY", "SOURCE"]).agg({"PRICE": ["mean"]})

PRICE
                      mean
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

The average earnings by COUNTRY, SOURCE, SEX, AGE grouping

In [13]:
df.groupby(["COUNTRY", "SOURCE", "SEX", "AGE"]).agg({"PRICE": "mean"}).head()

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897

We will sort the output by PRICE. To better see the output from the previous question, we will apply the sort_values method to PRICE in descending order. We will save the output as 'agg_df'.

In [14]:
agg_df = df.groupby(["COUNTRY","SOURCE","SEX","AGE"]).agg({"PRICE":"mean"}).sort_values("PRICE", ascending=False)
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


Let's convert the index names into variable names. All variables other than PRICE in the output of the third question are index names. Let's convert these names into variable names.

In [15]:
agg_df = agg_df.reset_index()
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


Let's convert the numerical variable AGE into a categorical variable and then add it to agg_df. We will create intervals for it.

In [16]:
agg_df['AGE_CAT'] = pd.cut(agg_df['AGE'], [0, 18, 23, 30, 40, 70], labels=['0_18', '19_23', '24_30', '31_40', '41_70'])
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
0,bra,android,male,46,59.0,41_70
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40


Let's define new level-based customers and add them as variables to the dataset.

- We will define a variable named 'customers_level_based' and later add this variable to the dataset.
- Note: After creating the values for 'customers_level_based' with list comprehensions, these values need to be deduplicated. For example, there may be multiple occurrences of expressions like 'USA_ANDROID_MALE_0_18.' These should be grouped and the price averages should be calculated.

In [17]:
agg_df["customers_level_based"] = [(i[0]+"_"+i[1]+"_"+i[2]+"_"+i[5]).upper() for i in agg_df.values]
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based
0,bra,android,male,46,59.0,41_70,BRA_ANDROID_MALE_41_70
1,usa,android,male,36,59.0,31_40,USA_ANDROID_MALE_31_40
2,fra,android,female,24,59.0,24_30,FRA_ANDROID_FEMALE_24_30
3,usa,ios,male,32,54.0,31_40,USA_IOS_MALE_31_40
4,deu,android,female,36,49.0,31_40,DEU_ANDROID_FEMALE_31_40


Let's segment the new customers (USA_ANDROID_MALE_0_18) into groups.

- Let's segment based on PRICE.
- Let's add the segments to agg_df with the name 'SEGMENT'.
- Describe the segments.

In [18]:
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4 ,["D","C","B","A"])
agg_df.groupby("SEGMENT").agg({"PRICE": ["mean","max","sum"]})

PRICE                        
              mean        max          sum
SEGMENT                                   
D        27.302596  31.105263  2375.325850
C        32.933339  34.000000  3128.667165
B        35.436170  37.000000  2870.329792
A        41.434736  59.000000  3521.952577

Let's classify the new incoming customers and estimate how much revenue they can potentially generate.

- Which segment does a 33-year-old Turkish woman using an ANDROID device belong to, and what is the expected average spending she can make?

In [19]:
new_user = 'TUR_ANDROID_FEMALE_31_40'
agg_df[agg_df["customers_level_based"] == new_user]

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based,SEGMENT
18,tur,android,female,32,43.000000,31_40,TUR_ANDROID_FEMALE_31_40,A
35,tur,android,female,31,40.666667,31_40,TUR_ANDROID_FEMALE_31_40,A


- Which segment does a 35-year-old French woman using an IOS device belong to, and what is the expected average spending she can make?

In [20]:
new_user = "FRA_IOS_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user]

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based,SEGMENT
208,fra,ios,female,40,33.000000,31_40,FRA_IOS_FEMALE_31_40,C
221,fra,ios,female,31,32.636364,31_40,FRA_IOS_FEMALE_31_40,C
